# Comment linéariser ?

## Introduction

Quand on optimise un problème, parfois, la fonction objectif, ou les contraintes peuvent contenir 
des éléments <b>non linéaires</b>.

On préfère souvent simplifier un problème non linéaire en problème linéaire, ce qui permet de résoudre l'optimisation plus facilement car le solveur calcule plus facilement si c'est linéaire. Seulement, il existe des <b>techniques</b>, fixées, par deux documentations et un livre en particulier, de linéarisation , à connaitre. Dans ce notebook on va comparer les méthodes trouvées dans les documents suivants :

- AIMMS Modeling Guide - Integer Programming Tricks
- Le cours Coursera Operations research - Coursera de Ling-Chieh Kung
Semaine 4 https://www.youtube.com/watch?v=csFOAOVEZzc
- Transformation and Linearization Techniques in Optimization:
A State-of-the-Art Survey de MDPI
- Le livre The Linearization Method
for Constrained Optimization de chez Springer

Dans ce notebook, on va voir comment détecter ces éléments non linéaires, 
et analyser les techniques fixes pour linéariser ces éléments.


## Précisions

Traduction du cours de 孔令傑副教授 - Taiwan University - Operations research - Coursera de Ling-Chieh Kung
Semaine 4 - Comment linéariser. Je vais au fil du temps y ajouter des analyses d'autres cours.
Ajout de tests perso avec les solveurs.

Etude fournie par Estelle Derrien - Github estellederrien

!! Création en cours - Très lourdes modifications à venir!!



# Sommaire

1. Exemples concrets.
    - A/ Réduction de coûts d'installation non linéaire.
    - B/ Augmentation de coûts d'installation non linéaire.
2. Quand la non linéarité apparait-elle ?
3. Objectif de la linéarisation.
4. Exemple introductif : linéariser une fonction de valeur absolue.
5. Linéariser les contraintes.
6. Linéariser la fonction objectif.
7. Linéariser la multiplication de 2 variables de décision.

# 1. Exemple concret 

## A/ Réduction de coûts d'installation non linéaire.

HIstoire :<br>
Une entreprise construit 2 produits qu'elle vends 10e et 12 e avec des ressources limitées.. <br>
Construire chaque produit implique un coût de préparation de 20e et 25e<br>
SI l'entreprise construit les 2 produits, cela provoque une réduction du coût de préparation de 10e<br>

Modélisation : 

Objectif <br>
Max (10 X1 + 12 X2 - 20 Z1 - 25 Z2 + 10 Z1*Z2)<br>
COntraintes en ressources diverses(Pas utiles de les préciser dans ce problème):<br>
2X1 + X2 <= 6<br>
X1 + 2X2 <= 8<br>

<b>

- On répère dans la fonction objectif que deux variables de décision binaires sont multipliées
, puis multipliées par 10 et additionnées dans la fonction objectif. 

- Alerte : Il s'agit d'un programme NON LINEAIRE ! Et pourtant, on va le linéariser, avec une 
première technique </b>

Source 
https://www.youtube.com/watch?v=csFOAOVEZzc

Scenario 1A

## Modélisation avec CPLEX

Déjà on crée le PL suivant : 
Une entreprise construit 2 produits qu'elle vends 10e et 12 e. <br>
Construire chaque produit implique un coût de préparation de 20e et 25e.<br>

On a donc enlevé la contrainte non linéaire 10 Z1*Z2<br>

On voit que Cplex nous donne une solution maximisée, il dit de fabriquer 4 objets X2, ce qui fait 12*4 = 48 euros de bénéfice
duquel il déduit correctement les 25 euros d'installation

In [25]:
import cplex
import docplex.mp
from docplex.mp.model import Model


# On crée notre modèle
model = Model(name='maximisation', log_output=True)

# On crée nos variables de décision
X1 = model.integer_var(name='X1')
X2 = model.integer_var(name='X2')
Z1 = model.binary_var(name='Z1')
Z2 = model.binary_var(name='Z2')

# On crée la fonction objectif
model.maximize( 10 * X1 + 12 * X2 - 20 * Z1 - 25 * Z2 )

# On crée les contraintes
model.add_constraint(2 * X1 + X2 <= 6)
model.add_constraint(X1 + 2 * X2 <= 8)

#if then constraint
model.add_constraint(model.if_then(X1 >= 1, Z1 == 1))
model.add_constraint(model.if_then(X2 >= 1, Z2 == 1))


model.parameters.mip.display.set(0)
sol_model = model.solve()
model.print_solution()

Version identifier: 22.1.1.0 | 2023-02-09 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Display                             0
objective: 23
status: OPTIMAL_SOLUTION(2)
  X2=4
  Z2=1


## Ajout de la contrainte non linéaire.

On va ajouter la contrainte non linéaire, on a donc pas encore linéarisé cette contrainte,  et voir si Cplex accepte quand même cette contrainte écrite en dur, ou si cela provoque une erreur.

On force la production de X2 à au moins 1 objet, de façon à activer nos deux contraintes binaires Z1 et Z2 à 1 et donc, à tester si 
10 * ( Z1 * Z2) va bien être activé et soustraire les 10 euros au profit global.

Résultat : 
On gagne 1 * 10 + 3 * 12 = 46 euros - 20 + 25 de coût d'installation) = 1 euros + 10 euros de réduction de coût d'installation parce que 
Z1 et Z2 sont activées , ce qui donne 11 euros de gains maximisés. 

On a pas d'erreur avec CPLEX, malgré que l'on a pas encore utilisé la technique de linéarisation dans le code.


In [26]:
import cplex
import docplex.mp
from docplex.mp.model import Model


# On crée notre modèle
model = Model(name='maximisation', log_output=True)

# On crée nos variables de décision
X1 = model.integer_var(name='X1')
X2 = model.integer_var(name='X2')
Z1 = model.binary_var(name='Z1')
Z2 = model.binary_var(name='Z2')

# On crée la fonction objectif
model.maximize( 10 * X1 + 12 * X2 - 20 * Z1 - 25 * Z2 + 10 * ( Z1 * Z2) )

# On crée les contraintes
model.add_constraint(2 * X1 + X2 <= 6)
model.add_constraint(X1 + 2 * X2 <= 8)
model.add_constraint(X1  >= 2) # Je force la production de X1 à au moins 2 objets pour qu'on voit la variable W s'activer
model.add_constraint(X2  >= 2) # Je force la production de X2 à au moins 2 objets pour qu'on voit la variable W s'activer

# if then constraint
model.add_constraint(model.if_then(X1 >= 1, Z1 == 1))
model.add_constraint(model.if_then(X2 >= 1, Z2 == 1))


model.parameters.mip.display.set(0)
sol_model = model.solve()
model.print_solution()

Version identifier: 22.1.1.0 | 2023-02-09 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Display                             0
objective: 9
status: OPTIMAL_SOLUTION(2)
  X1=2
  X2=2
  Z1=1
  Z2=1


## Linéarisation de la contrainte non linéaire.

Ok, donc CPLEX a accepté que j'écrive directement la contrainte non linéaire dans la fonction objectif.
Maintenant, on va réécrire le code en linéarisant comme dans la vidéo de Taîwan University, et voir si on trouve pareil.

Cette première technique dit qu'on remplace la multiplication des 2 vars binaires Z1 * Z2 par la nouvelle variable de décision binaire W
que l'on crée.


On voit qu'on trouve bien la même solution, avec la linéarisation appliquée. On voit que la variable W s'est bien retrouvée à 1 quand X1 et X2 sont produits, et que la réduction de prix a été correctement appliquée.

In [27]:
import cplex
import docplex.mp
from docplex.mp.model import Model


# On crée notre modèle
model = Model(name='maximisation', log_output=True)

# On crée nos variables de décision
X1 = model.integer_var(name='X1')
X2 = model.integer_var(name='X2')
Z1 = model.binary_var(name='Z1')
Z2 = model.binary_var(name='Z2')
W = model.binary_var(name='W')

# On crée la fonction objectif
model.maximize( 10 * X1 + 12 * X2 - 20 * Z1 - 25 * Z2 + 10 * W )

# On crée les contraintes
model.add_constraint(2 * X1 + X2 <= 6)
model.add_constraint(X1 + 2 * X2 <= 8)
model.add_constraint(X1  >= 2) # Je force la production de X1 à au moins 2 objets pour qu'on voit la variable W s'activer
model.add_constraint(X2  >= 2) # Je force la production de X2 à au moins 2 objets pour qu'on voit la variable W s'activer

# On gère la nouvelle variable W
model.add_constraint(W  <= Z1)
model.add_constraint(W  <= Z2)

# if then constraint
model.add_constraint(model.if_then(X1 >= 1, Z1 == 1))
model.add_constraint(model.if_then(X2 >= 1, Z2 == 1))


model.parameters.mip.display.set(0)
sol_model = model.solve()
model.print_solution()

Version identifier: 22.1.1.0 | 2023-02-09 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Display                             0
objective: 9
status: OPTIMAL_SOLUTION(2)
  X1=2
  X2=2
  Z1=1
  Z2=1
  W=1


## Avec Pulp, le solveur Linéaire

Vu qu'on a linéarisé notre fonction objectif, on peut désormais essayer de résoudre avec un solveur linéaire et voir si on trouve le même résultat.
Testons déjà le code Pulp pour voir si il trouve pareil que Cplex, sans la contrainte non linéaire 10 * Z1 * Z2 :

Oui, donc il trouve pareil, pour l'instant

In [28]:
# Importer la librairie Pulp 
from pulp import *

# Créer un programme linéaire de maximisation
Mon_Probleme = LpProblem('maximisation', LpMaximize)  

# Créer les variables du problème 
X1 = LpVariable("X1", 0, None, LpInteger)   
X2 = LpVariable("X2", 0, None, LpInteger)   
Z1 = LpVariable("Z1", 0, None, LpBinary) 
Z2 = LpVariable("Z2", 0, None, LpBinary) 


# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
Mon_Probleme += ( 10 * X1 + 12 * X2 - 20 * Z1 - 25 * Z2  )

# Les contraintes : 

Mon_Probleme += 2 * X1 + X2 <= 6
Mon_Probleme += X1 + 2 * X2 <= 8

# if then constraint
M = 1000  # M se calcule selon une certaine méthode ( BigM)

# si X1 > 1 alors Z1 = 1 s'écrit comme cela avec PULP
Mon_Probleme += Z1  >= (X1 - 1 )/M
# si X2 > 1 alors Z2 = 2 s'écrit comme cela avec PULP
Mon_Probleme += Z2  >= (X2 - 1 )/M


# Résoudre
Mon_Probleme.solve()
# On imprime les variables qui ont leur valeur optimisées
for v in Mon_Probleme.variables():
    print(v.name, "=", v.varValue)
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", value(Mon_Probleme.objective))

X1 = 0.0
X2 = 4.0
Z1 = 0.0
Z2 = 1.0
Profit total maximisé =  23.0


## On essaye d'ajouter la contrainte linéarisée, avec PULP

On voit que Pulp trouve la même solution que CPLEX et qu'il effectue bien la réduction de coût lorsque la variable W est activée.
Cependant, méfiance, parce que quand je force X1 et X2 à 1 objet à produire, les 2 solveurs ne proposent pas la même solution.

In [29]:
# Importer la librairie Pulp 
from pulp import *

# Créer un programme linéaire de maximisation
Mon_Probleme = LpProblem('maximisation', LpMaximize)  

# Créer les variables du problème 
X1 = LpVariable("X1", 0, None, LpInteger)   
X2 = LpVariable("X2", 0, None, LpInteger)   
Z1 = LpVariable("Z1", 0, None, LpBinary) 
Z2 = LpVariable("Z2", 0, None, LpBinary) 
W = LpVariable("W", 0, None, LpBinary) 

# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
Mon_Probleme += ( 10 * X1 + 12 * X2 - 20 * Z1 - 25 * Z2 + 10 * W )

# Les contraintes : 

Mon_Probleme += 2 * X1 + X2 <= 6
Mon_Probleme += X1 + 2 * X2 <= 8

Mon_Probleme += X1 >= 2 # Je force la production de X1 à au moins 2 objets pour qu'on voit la variable W s'activer
Mon_Probleme += X2 >= 2 # Je force la production de X1 à au moins 2 objets pour qu'on voit la variable W s'activer


# On gère la nouvelle variable W
Mon_Probleme += W <= Z1
Mon_Probleme += W <= Z2

# if then constraint
M = 1000  # M se calcule selon une certaine méthode ( BigM)

# si X1 > 1 alors Z1 = 1 s'écrit comme cela avec PULP
Mon_Probleme += Z1  >= (X1 - 1 )/M
# si X2 > 1 alors Z2 = 2 s'écrit comme cela avec PULP
Mon_Probleme += Z2  >= (X2 - 1 )/M


# Résoudre
Mon_Probleme.solve()
# On imprime les variables qui ont leur valeur optimisées
for v in Mon_Probleme.variables():
    print(v.name, "=", v.varValue)
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", value(Mon_Probleme.objective))

W = 1.0
X1 = 2.0
X2 = 2.0
Z1 = 1.0
Z2 = 1.0
Profit total maximisé =  9.0


## B/ Augmentation de coûts d'installation non linéaire.

HIstoire :<br>
Une entreprise construit 2 produits qu'elle vends 10e et 12e avec des ressources limitées. <br>
Construire chaque produit implique un coût de préparation de 20e et 25e<br>
SI l'entreprise construit les 2 produits, cela provoque une augmentation du coût de préparation de 10e<br>

Modélisation : 

Objectif <br>
Max (10 X1 + 12 X2 - 20 Z1 - 25 Z2 - 10 Z1*Z2)<br>
COntraintes ressources diverses(Pas utiles de les préciser dans ce problème):<br>
2X1 + X2 <= 6<br>
X1 + 2X2 <= 8<br>

<b>

- On répère dans la fonction objectif que deux variables de décision binaires sont multipliées, 
et cette fois ci, elles sont soustraites.

- Alerte : Il s'agit d'un programme NON LINEAIRE ! Et pourtant, on va le linéariser, avec une 
seconde  technique </b>

## Solution avec CPLEX, en linéarisant directement

Ici, pour linéariser  Z1 * Z2, on remplace Z1 * Z2 par la variable de décision W
Mais attention, vu que c'est soustrait dans la fonction objectif, la méthode n'est pas la même


In [30]:
import cplex
import docplex.mp
from docplex.mp.model import Model


# On crée notre modèle
model = Model(name='maximisation', log_output=True)

# On crée nos variables de décision
X1 = model.integer_var(name='X1')
X2 = model.integer_var(name='X2')
Z1 = model.binary_var(name='Z1')
Z2 = model.binary_var(name='Z2')
W = model.binary_var(name='W')

# On crée la fonction objectif
# Rappel on a linéarisé la fonction objectif suivante : 
# model.maximize( 10 * X1 + 12 * X2 - 20 * Z1 - 25 * Z2 - 10 * Z1 * Z2 )

model.maximize( 10 * X1 + 12 * X2 - 20 * Z1 - 25 * Z2 - 10 * W )

# On crée les contraintes
model.add_constraint(2 * X1 + X2 <= 600) # Je mets plus de ressources, sinon l'objectif est négatif avec 6
model.add_constraint(X1 + 2 * X2 <= 800)  # Je mets plus de ressources, sinon l'objectif est négatif avec 8
model.add_constraint(X1  >= 1) # Je force la production de X1 à au moins 1 objets pour qu'on voit la variable W s'activer
model.add_constraint(X2  >= 1) # Je force la production de X2 à au moins 1 objets pour qu'on voit la variable W s'activer


# On gère la nouvelle variable W qui a été linéarisée
model.add_constraint(W  >= Z1 + Z2 - 1)


# if then constraint
model.add_constraint(model.if_then(X1 >= 1, Z1 == 1))
model.add_constraint(model.if_then(X2 >= 1, Z2 == 1))


model.parameters.mip.display.set(0)
sol_model = model.solve()
model.print_solution()

Version identifier: 22.1.1.0 | 2023-02-09 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Display                             0
objective: 5273
status: OPTIMAL_SOLUTION(2)
  X1=132
  X2=334
  Z1=1
  Z2=1
  W=1


## Test avec Python Pulp

On fait le même modèle et on linéarise la fonction objectif, on peut donc en toute logique le solver avec Python Pulp, le solveurs linéaire.

Rappel : On crée une nouvelle variable binaire W, et on ajoute un code spécial dans les contraintes, pour remplacer Z1 * Z2 qui était non linéaire.

On voit qu'on trouve le même résultat qu'avec CPLEX.


In [31]:
# Importer la librairie Pulp 
from pulp import *

# Créer un programme linéaire de maximisation
Mon_Probleme = LpProblem('maximisation', LpMaximize)  

# Créer les variables du problème 
X1 = LpVariable("X1", 0, None, LpInteger)   
X2 = LpVariable("X2", 0, None, LpInteger)   
Z1 = LpVariable("Z1", 0, None, LpBinary) 
Z2 = LpVariable("Z2", 0, None, LpBinary) 
W = LpVariable("W", 0, None, LpBinary) 

# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
# Rappel on a linéarisé la fonction objectif suivante : 
# ( 10 * X1 + 12 * X2 - 20 * Z1 - 25 * Z2 - 10 * Z1 * Z2 )
Mon_Probleme += ( 10 * X1 + 12 * X2 - 20 * Z1 - 25 * Z2 - 10 * W )

# Les contraintes : 

Mon_Probleme += 2 * X1 + X2 <= 600
Mon_Probleme += X1 + 2 * X2 <= 800

Mon_Probleme += X1 >= 2 # Je force la production de X1 à au moins 2 objets pour qu'on voit la variable W s'activer
Mon_Probleme += X2 >= 2 # Je force la production de X1 à au moins 2 objets pour qu'on voit la variable W s'activer


# On gère la nouvelle variable W
Mon_Probleme += W  >= Z1 + Z2 - 1

# if then constraint
M = 1000  # M se calcule selon une certaine méthode ( BigM)

# si X1 > 1 alors Z1 = 1 s'écrit comme cela avec PULP
Mon_Probleme += Z1  >= (X1 - 1 )/M
# si X2 > 1 alors Z2 = 2 s'écrit comme cela avec PULP
Mon_Probleme += Z2  >= (X2 - 1 )/M


# Résoudre
Mon_Probleme.solve()
# On imprime les variables qui ont leur valeur optimisées
for v in Mon_Probleme.variables():
    print(v.name, "=", v.varValue)
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", value(Mon_Probleme.objective))

W = 1.0
X1 = 132.0
X2 = 334.0
Z1 = 1.0
Z2 = 1.0
Profit total maximisé =  5273.0


# 2. Quand la non linéarité apparait-elle ?

Résumé :

Dès que 
- 2 variables de décision, même différentes sont mutlipliées
- 1 variable de décision est élevée au carré
- 1 variable de décision est à la valeur absolue.


Exemple:


Prenons ce P.N.L, on voit que des coût sont soustraits si la variable binaire z est = à 1.

- max 50 z - ( 10 x1 + 12 x2 )z 
- s.t
- 2x1 + x2 >= 6
- x1 + 2x2 >= 8
- x1,x2 >= 0 
- z ∈ {0,1}

On ne le voit pas trop, mais la non linéarité apparait parce que 10x1 et 12x2 sont multipliés par z.
Autrement dit, dès que 2 variables de décisions sont multipliées, on trouve un cas de non linéarité, <b> même si ce sont 2 variables de décision différentes.</b>


# 2. Objectif de la linéarisation?

Linéariser un programme non linéaire permet d'utiliser un solveur linéaire à la place d'un solveur non linéaire, et c'est plus facile pour celui-ci de réaliser des calculs.

On ne peut pas tout linéariser, c'est selon les cas.

# 3. Exemple : linéariser une fonction de valeur absolue.

Supposons que nous souhaitons partager 1000$ entre 2 personnes de façon équitable.

On adopte cette méthode : plus petite est la différence entre les 2 montants, plus c'est équitable.

De toutes évidence, la réponse est 500$.

Pourrait-on créer un programme linéaire pour résoudre ce problème ?

Si on modélise comme ça :

- min (x2 - x1)
- s.t
- x1 + x2 = 1000
- xi >= 0 ∀i = 1,2

ca ne va pas, par ce que le résultat de la fonction objectif sera - 1000 car elle est minimisée
x1 sera = à 1000 et x2 à 0.

Si on modélise comme ça avec la fonction absolue, on pourrait se dire que cela va forcer la positivité:

- min |(x2 - x1)|
- s.t
- x1 + x2 = 1000
- xi >= 0 ∀i = 1,2

Mais cela ne va pas parce que la fonction est alors non linéaire. 

On sait que |x2 - x1| = max{x2 - x1, x1 - x2}
et w >=  max{x2 - x1, x1 - x2} <=> w >= x2 - x1 et w >= x1 - x2

On peut donc réécrire le p.l :
- min w
- s.t
- w >= x2 - x1
- w >= x1 - x2
- x1 + x2 = 1000
- xi >= 0 ∀i = 1,2








# 4. Linéariser les contraintes



# 5. Linéariser la fonction objectif

# 6. Linéariser la multiplication de 2 variables de décision.

In [32]:
# A VENIR JUIN 2023 !!!

# 7. Liens 

Lien : 
https://www.coursera.org/learn/operations-research-modeling/lecture/UHdtK/4-7-linearizing-max-min-functions

Lien d'un autre cours : 

https://www.adrian.idv.hk/2012-05-22-linearization/

https://www.mdpi.com/2227-7390/10/2/283